Giả sử bạn dự đoán có đậu phỏng vấn không dựa trên:

Số năm kinh nghiệm
Điểm phỏng vấn kỹ thuật
→ Logistic Regression học được:

"Nếu kinh nghiệm ≥ 3 năm và điểm ≥ 7 → xác suất đậu ~90%" 

→ Nó không vẽ đường thẳng phân chia cứng, mà ước lượng xác suất mềm.

✅ Ưu điểm:
Đơn giản, nhanh, dễ hiểu.
Cho xác suất đầu ra (rất hữu ích!).
Làm nền cho mạng neural (perceptron = logistic regression + hidden layer).
❌ Hạn chế:
Chỉ hiệu quả khi mối quan hệ gần tuyến tính.
Không xử lý tốt đặc trưng phi tuyến (trừ khi bạn tự tạo đặc trưng mới).


Logistic Regression = hồi quy tuyến tính + hàm sigmoid → phân loại nhị phân.
Không cần công thức phức tạp — chỉ cần hiểu: kết hợp đặc trưng → điểm số → xác suất.

In [5]:
import numpy as np

class SimpleLogisticRegression:
    def __init__(self, learning_rate=0.1, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def _sigmoid(self, z):
        # Hàm sigmoid: biến số thực thành xác suất [0, 1]
        # Tránh overflow bằng cách giới hạn z
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Khởi tạo trọng số ngẫu nhiên nhỏ
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Gradient Descent: lặp để cải thiện mô hình
        for _ in range(self.n_iters):
            # Dự đoán xác suất
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self._sigmoid(linear_model)

            # Tính gradient (hướng đi để giảm lỗi)
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            # Cập nhật trọng số
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict_proba(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        return self._sigmoid(linear_model)

    def predict(self, X):
        return (self.predict_proba(X) > 0.5).astype(int)

# --- Thử nghiệm với dữ liệu thật ---
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Tạo dữ liệu nhị phân đơn giản
X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0, n_informative=2,
                           n_clusters_per_class=1, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Huấn luyện mô hình tự code
model = SimpleLogisticRegression(learning_rate=0.1, n_iters=1000)
model.fit(X_train, y_train)

# Dự đoán
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"✅ Độ chính xác (tự code): {acc:.2%}")

# So sánh với sklearn (để kiểm tra)
from sklearn.linear_model import LogisticRegression as SklearnLR
sk_model = SklearnLR()
sk_model.fit(X_train, y_train)
sk_acc = sk_model.score(X_test, y_test)
print(f"🔍 Sklearn độ chính xác: {sk_acc:.2%}")

✅ Độ chính xác (tự code): 88.00%
🔍 Sklearn độ chính xác: 88.33%


🔹 3. Khi nào dùng Logistic Regression?
Bạn cần
xác suất đầu ra
(không chỉ nhãn)
Dữ liệu có
mối quan hệ phi tuyến phức tạp
(ví dụ: hình tròn lồng nhau)
Dữ liệu
ít nhiễu
,
gần phân tách tuyến tính
Số đặc trưng
rất lớn
mà không có regularization (dễ overfit)
Bạn cần mô hình
đơn giản, giải thích được
(trọng số = mức độ ảnh hưởng)
Bài toán
ảnh, âm thanh, văn bản thô
(hãy dùng CNN/RNN thay vì LR)
Làm
baseline
trước khi thử mô hình phức tạp
Cần
độ chính xác cực cao
trên dữ liệu phi tuyến → dùng XGBoost hoặc Neural Net
